# Pandas Groupby運用

In [1]:
# 載入套件
import numpy as np
import pandas as pd
import pyodbc

In [3]:
# 連結資料庫
conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=(local); DATABASE=MLDATASET')
SQLSTR = 'SELECT * FROM BANK'

In [4]:
# 讀取資料
bank = pd.read_sql(con=conn, sql = SQLSTR)

In [6]:
df = bank.copy()

In [7]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602.0,Hargrave,619.0,France,Female,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0
1,15647311.0,Hill,608.0,Spain,Female,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0
2,15619304.0,Onio,502.0,France,Female,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0
3,15701354.0,Boni,699.0,France,Female,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0
4,15737888.0,Mitchell,850.0,Spain,Female,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0


In [53]:
# 一般寫法(一)
df.groupby(['Gender']).agg({'Gender':'count'})

,Gender
Gender,
Female,4543
Male,5457


In [70]:
# 一般寫法(二)
df.groupby(['Gender'])[['Gender']].count()

,Gender
Gender,
Female,4543
Male,5457


In [178]:
# 一般寫法(三)
df.groupby('Gender')['Age'].agg([np.sum, np.mean, np.median, np.min, np.max, np.std,
                                 lambda x : np.quantile(x, q = 0.25), 
                                 lambda x : np.quantile(x, q = 0.5),
                                 lambda x : np.percentile(x, q = 75)])

,sum,mean,median,amin,amax,std,<lambda_0>,<lambda_1>,<lambda_2>
Gender,,,,,,,,,
Female,178260.0,39.238389,38.0,18.0,85.0,10.588588,32.0,38.0,45.0
Male,210958.0,38.658237,37.0,18.0,92.0,10.396780,32.0,37.0,43.0


In [ ]:
########################################################################################

In [82]:
# 複合變數
df.groupby(['Gender']).agg({'CustomerId':'count',
                            'Age':'mean',
                            'Balance':'median'})

,CustomerId,Age,Balance
Gender,,,
Female,4543,39.238389,96147.55
Male,5457,38.658237,98064.97


In [97]:
# 複合變數(一個量值變數，不同變化：平均數、中位數、變異數、標準差)
df.groupby(['Gender']).agg({'CustomerId':'count',
                            'Age':['mean','median','var','std'],
                            'Balance':'median'})

CustomerId        Age                                 Balance
            count       mean median         var        std    median
Gender                                                              
Female       4543  39.238389   38.0  112.118191  10.588588  96147.55
Male         5457  38.658237   37.0  108.093037  10.396780  98064.97

In [191]:
# 複合變數(一個量值變數，不同變化：平均數、四分位數(Q1, Q2, Q3))
df.groupby(['Geography', 'Gender']).agg({'CustomerId':'count',
                                         'Age':[np.mean, np.median, np.min, np.max, np.std],
                                         'Balance':[np.mean, np.median, lambda x:np.percentile(x, q = 25),
                                                                        lambda x:np.percentile(x, q = 50),
                                                                        lambda x:np.quantile(x, q = 0.75),
                                                    np.std],
                                         'CreditScore':np.mean})

CustomerId        Age                                \
                      count       mean median  amin  amax        std   
Geography Gender                                                       
France    Female       2261  38.773994   37.0  18.0  85.0  10.685519   
          Male         2753  38.296404   37.0  18.0  92.0  10.285569   
Germany   Female       1193  40.154233   39.0  18.0  77.0  10.461002   
          Male         1316  39.424772   38.0  18.0  84.0  10.563570   
Spain     Female       1089  39.199265   38.0  19.0  79.0  10.469659   
          Male         1388  38.649135   37.0  18.0  88.0  10.425004   

                        Balance                                       \
                           mean      median   <lambda_0>  <lambda_1>   
Geography Gender                                                       
France    Female   60322.670159   47536.400       0.0000   47536.400   
          Male     63546.284875   69413.440       0.0000   69413.440   
Germany   Female  119145.966471  118907.600  102286.6000  118907.600   
          Male    120259.668222  120120.490  103397.7475  120120.490   
Spain     Female   59862.092534       0.000       0.0000       0.000   
          Male     63352.833746   69857.005       0.0000   69857.005   

                                            CreditScore  
                   <lambda_2>           std        mean  
Geography Gender                                         
France    Female  119013.8600  63576.547900  649.185759  
          Male    122806.9500  64562.683717  650.064657  
Germany   Female  137326.6500  26340.359839  653.093881  
          Male    137815.0700  27624.732304  649.966565  
Spain     Female  119799.2700  64280.969774  651.769513  
          Male    121348.4075  64181.321963  650.992075

In [ ]:
###############################################################################3

In [192]:
# 四分位其它寫法(四分位數會放置ROW位置)
df.groupby(['Gender'])[['Age', 'Balance']].quantile([0, 0.25, 0.5, 0.75])

Age     Balance
Gender                       
Female 0.00  18.0       0.000
       0.25  32.0       0.000
       0.50  38.0   96147.550
       0.75  45.0  127141.785
Male   0.00  18.0       0.000
       0.25  32.0       0.000
       0.50  37.0   98064.970
       0.75  43.0  127900.030

In [ ]:
########################################################################################

In [119]:
# APPLY運用(一)
df[['CreditScore', 'Age']].head().apply(np.sum, axis = 1)

0    661.0
1    649.0
2    544.0
3    738.0
4    893.0
dtype: float64

In [121]:
# APPLY運用(二)
df[['CreditScore', 'Age']].head().apply(lambda x:x/10, axis = 1)

,CreditScore,Age
0,61.9,4.2
1,60.8,4.1
2,50.2,4.2
3,69.9,3.9
4,85.0,4.3


In [131]:
# APPLY運用(三)
def test1(x):
    x = x/10
    return x

df[['CreditScore', 'Age']].head().apply(test1, axis = 1)

,CreditScore,Age
0,61.9,4.2
1,60.8,4.1
2,50.2,4.2
3,69.9,3.9
4,85.0,4.3


In [ ]:
##################################################################################

In [193]:
# pivot & pivot_table 寫法

In [202]:
# pivot 資料
import seaborn as sns
air = sns.load_dataset('flights')

In [203]:
air

,year,month,passengers
0,1949,Jan,112
1,1949,Feb,118
2,1949,Mar,132
3,1949,Apr,129
4,1949,May,121
...,...,...,...
139,1960,Aug,606
140,1960,Sep,508
141,1960,Oct,461
142,1960,Nov,390


In [207]:
# pivot 三個重要參數：index-row, columns-columns, values-需放入的量值變數
air.pivot(index = 'month', columns = 'year', values = 'passengers')

year,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960
month,,,,,,,,,,,,
Jan,112,115,145,171,196,204,242,284,315,340,360,417
Feb,118,126,150,180,196,188,233,277,301,318,342,391
Mar,132,141,178,193,236,235,267,317,356,362,406,419
Apr,129,135,163,181,235,227,269,313,348,348,396,461
May,121,125,172,183,229,234,270,318,355,363,420,472
Jun,135,149,178,218,243,264,315,374,422,435,472,535
Jul,148,170,199,230,264,302,364,413,465,491,548,622
Aug,148,170,199,242,272,293,347,405,467,505,559,606
Sep,136,158,184,209,237,259,312,355,404,404,463,508


In [211]:
# 有些資料會有「重複值」所以必需用pivot_table
df2 = df[['Gender', 'Geography', 'CreditScore']] # 建立示範資料

In [220]:
# 參數與pivot類似，最重要差在 aggfunc 這項，該變數決定重復的值是要用何種方式計算(ex：加總、平均…)
print('加總')
print(df2.pivot_table(index='Gender', columns='Geography', aggfunc=np.sum)) # 加總
print('\n')

print('平均')
print(df2.pivot_table(index='Gender', columns='Geography', aggfunc=np.mean)) # 平均
print('\n')

print('中位數')
print(df2.pivot_table(index='Gender', columns='Geography', aggfunc=np.median)) # 中位數
print('\n')

加總
          CreditScore                    
Geography      France   Germany     Spain
Gender                                   
Female      1467809.0  779141.0  709777.0
Male        1789628.0  855356.0  903577.0


平均
          CreditScore                        
Geography      France     Germany       Spain
Gender                                       
Female     649.185759  653.093881  651.769513
Male       650.064657  649.966565  650.992075


中位數
          CreditScore               
Geography      France Germany  Spain
Gender                              
Female          652.0   651.0  653.0
Male            653.0   650.5  650.0




In [93]:
# 匿名函數寫法
(lambda x : x * 2)(2)

4

In [ ]:
# apply運用資料來源
https://zhuanlan.zhihu.com/p/100064394
    
# pivot, pivot_table 運用資料來源    
https://blog.csdn.net/liuweiyuxiang/article/details/78220530    